# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [1]:
# import libraries here

import json
import requests
import pandas as pd

Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [2]:
# your code here

username = 'leonvanderlee'
token = 'aceb352de54ed83918f931fc5de1920cf4c396ef'

response = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks', auth=(username,token))

data = pd.DataFrame(response.json())

data.head()

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,mirror_url,archived,disabled,open_issues_count,license,forks,open_issues,watchers,default_branch,permissions
0,286608878,MDEwOlJlcG9zaXRvcnkyODY2MDg4Nzg=,dataV2-labs,joaopq/dataV2-labs,False,"{'login': 'joaopq', 'id': 63979330, 'node_id':...",https://github.com/joaopq/dataV2-labs,None,True,https://api.github.com/repos/joaopq/dataV2-labs,...,None,False,False,0,None,0,0,0,master,"{'admin': False, 'push': False, 'pull': True}"
1,286525274,MDEwOlJlcG9zaXRvcnkyODY1MjUyNzQ=,dataV2-labs,JoanRBC/dataV2-labs,False,"{'login': 'JoanRBC', 'id': 60225388, 'node_id'...",https://github.com/JoanRBC/dataV2-labs,None,True,https://api.github.com/repos/JoanRBC/dataV2-labs,...,None,False,False,0,None,0,0,0,master,"{'admin': False, 'push': False, 'pull': True}"
2,286521316,MDEwOlJlcG9zaXRvcnkyODY1MjEzMTY=,dataV2-labs,rafaelmello82/dataV2-labs,False,"{'login': 'rafaelmello82', 'id': 64922281, 'no...",https://github.com/rafaelmello82/dataV2-labs,None,True,https://api.github.com/repos/rafaelmello82/dat...,...,None,False,False,0,None,0,0,0,master,"{'admin': False, 'push': False, 'pull': True}"
3,286520497,MDEwOlJlcG9zaXRvcnkyODY1MjA0OTc=,dataV2-labs,cfmago/dataV2-labs,False,"{'login': 'cfmago', 'id': 36172682, 'node_id':...",https://github.com/cfmago/dataV2-labs,None,True,https://api.github.com/repos/cfmago/dataV2-labs,...,None,False,False,0,None,0,0,0,master,"{'admin': False, 'push': False, 'pull': True}"
4,286517824,MDEwOlJlcG9zaXRvcnkyODY1MTc4MjQ=,dataV2-labs,jinverno/dataV2-labs,False,"{'login': 'jinverno', 'id': 65239409, 'node_id...",https://github.com/jinverno/dataV2-labs,None,True,https://api.github.com/repos/jinverno/dataV2-labs,...,None,False,False,0,None,0,0,0,master,"{'admin': False, 'push': False, 'pull': True}"


#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [3]:
# your code here

languages = data['language'].unique()

languages

array([None, 'Jupyter Notebook'], dtype=object)

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [4]:
# your code here

commits = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/commits?q=date:2018-10-01..2018-10-31', auth=(username,token))

commits = pd.DataFrame(commits.json())

commits.head()

,sha,node_id,commit,url,html_url,comments_url,author,committer,parents
0,3050baad4bdf4ee65be7b6d56ee4902abf59d5fb,MDY6Q29tbWl0MjEzNjQxMTcwOjMwNTBiYWFkNGJkZjRlZT...,"{'author': {'name': 'kelsey-ironhack', 'email'...",https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/dataV2-la...,https://api.github.com/repos/ironhack-datalabs...,"{'login': 'kelsey-ironhack', 'id': 52278191, '...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '680a9ca1cebd77383a8a104ab48c868d1600...
1,680a9ca1cebd77383a8a104ab48c868d1600790f,MDY6Q29tbWl0MjEzNjQxMTcwOjY4MGE5Y2ExY2ViZDc3Mz...,"{'author': {'name': 'kelsey-ironhack', 'email'...",https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/dataV2-la...,https://api.github.com/repos/ironhack-datalabs...,"{'login': 'kelsey-ironhack', 'id': 52278191, '...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '1421d2b97b1e250bbd1c321e7c38a1ef87fd...
2,1421d2b97b1e250bbd1c321e7c38a1ef87fd2c49,MDY6Q29tbWl0MjEzNjQxMTcwOjE0MjFkMmI5N2IxZTI1MG...,"{'author': {'name': 'kelsey-ironhack', 'email'...",https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/dataV2-la...,https://api.github.com/repos/ironhack-datalabs...,"{'login': 'kelsey-ironhack', 'id': 52278191, '...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '023a27f4c4bb515a77bfda08f9309c1b1e33...
3,023a27f4c4bb515a77bfda08f9309c1b1e333e72,MDY6Q29tbWl0MjEzNjQxMTcwOjAyM2EyN2Y0YzRiYjUxNW...,"{'author': {'name': 'kbartonhenry', 'email': '...",https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/dataV2-la...,https://api.github.com/repos/ironhack-datalabs...,"{'login': 'kbartonhenry', 'id': 22545329, 'nod...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '574666b2fc86a71f60f334545fe4d16ab61a...
4,574666b2fc86a71f60f334545fe4d16ab61ab28f,MDY6Q29tbWl0MjEzNjQxMTcwOjU3NDY2NmIyZmM4NmE3MW...,"{'author': {'name': 'kbartonhenry', 'email': '...",https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/dataV2-la...,https://api.github.com/repos/ironhack-datalabs...,"{'login': 'kbartonhenry', 'id': 22545329, 'nod...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '6f83d1446dbb6bd600a2082e8147b79e420f...


#### 2. Count how many commit objects are contained in the array.

In [5]:
# your code here

commit = commits['commit'].count()

commit

30

## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [6]:
# your code here

# username = 'leonvanderlee'
# token = 'aceb352de54ed83918f931fc5de1920cf4c396ef'

# response2 = requests.get('https://api.github.com/repos/ironhack-datalabs/scavenger/forks', auth=(username,token))

# data2 = pd.DataFrame(response2.json())

# data2.head()


username = 'leonvanderlee'
token = 'aceb352de54ed83918f931fc5de1920cf4c396ef'

response2 = requests.get('https://api.github.com/repos/ironhack-datalabs/scavenger/contents', auth=(username,token))

data2 = pd.DataFrame(response2.json())

data2.head()

,name,path,sha,size,url,html_url,git_url,download_url,type,_links
0,.gitignore,.gitignore,e43b0f988953ae3a84b00331d0ccf5f7d51cb3cf,10,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,https://raw.githubusercontent.com/ironhack-dat...,file,{'self': 'https://api.github.com/repos/ironhac...
1,15024,15024,2945e51c87ad5da893c954afcf092f06343bbb7d,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,{'self': 'https://api.github.com/repos/ironhac...
2,15534,15534,5af6f2a7287e4191f39e55693fc1e9c8918d1d3a,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,{'self': 'https://api.github.com/repos/ironhac...
3,17020,17020,9c49f920aa4d9433fa99a5824128f0e6b90ec5f2,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,{'self': 'https://api.github.com/repos/ironhac...
4,30351,30351,c488d7f64088c852e22067d48fdc64ee3670f3ba,0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,None,dir,{'self': 'https://api.github.com/repos/ironhac...


#### 2.  Sort the filenames ascendingly.

In [7]:
# your code here

filenames = data2['name'].unique()

filenames

array(['.gitignore', '15024', '15534', '17020', '30351', '40303', '44639',
       '45525', '47222', '47830', '49418', '50896', '55417', '55685',
       '60224', '64880', '66032', '68848', '70751', '70985', '88596',
       '89046', '89338', '91701', '97881', '98750'], dtype=object)

#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [8]:
# your code here

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [9]:
# your code here

#### 5. Print out the joke.

In [10]:
# your code here